In [1]:
!pip install datasets transformers nltk rouge_score nvidia-ml-py3 evaluate

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=09e94582be0783d2e8777395a4b35bf5c70d65a2967fa4af4a59f6f6a2164332
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19190 sha256=736ea946a296ddaf3e00f6b9a99cd86a78e06d15212e222518054d7a2ba7dda8
  Stored in directory: /root/.cache/pip/wheels/74/d2/c1/2ea351258984f451bd34e5ff2928621ab030e2eda5ffd2fdec
Successfully built rouge_score nvidia-ml-py3


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from datasets import load_dataset

# 50k training samples, 12.5k validation samples
dataset = load_dataset("ccdv/arxiv-summarization", split=['train[:50000]', 'train[50000:62500]'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset arxiv_summarization_dataset downloaded and prepared to /root/.cache/huggingface/datasets/ccdv___arxiv_summarization_dataset/section/1.0.0/fa2c9abf4312afb8660ef8e041d576b8e3943ea96ae771bd3cd091b5798e7cc3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from datasets import DatasetDict

small_dataset = DatasetDict()
small_dataset["train"] = dataset[0]
small_dataset["validation"] = dataset[1]

small_dataset_2 = DatasetDict({
    'train': dataset[0],
    'validation': dataset[1]
})

In [5]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [6]:
# original pretrained model
model_checkpoint = "facebook/bart-base"
# model finetuned by us
to_resume_model_checkpoint = 'adityashukzy/bart-base-arxiv-sum'
# model name of model finetuned by us
model_name = f"{model_checkpoint.split('/')[-1]}-arxiv-sum"

In [7]:
""" Step 1: Load BartTokenizerFast to tokenize & encode the texts. Pad to the right """
from transformers import AutoTokenizer, BartTokenizerFast

tokenizer = AutoTokenizer.from_pretrained(to_resume_model_checkpoint)

In [8]:
""" Step 2: Truncate each document to 2048 tokens and each abstract to 256 tokens """

max_input_length = 512
max_target_length = 256

""" Step 3: Define preprocessing function to tokenize, truncate and arrange data properly """

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["abstract"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
""" Step 4: Map preprocess function to each sample from data """

tokenized_datasets = small_dataset.map(preprocess_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

In [11]:
""" Step 1: Load the model (BartForConditionalGeneration) """

from transformers import AutoModelForSeq2SeqLM, BartForConditionalGeneration

model = AutoModelForSeq2SeqLM.from_pretrained(to_resume_model_checkpoint)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
""" Step 2: Set arguments & hyperparameters """

from transformers import Seq2SeqTrainingArguments

batch_size = 64
num_train_epochs = 10
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size

args = Seq2SeqTrainingArguments(
    output_dir=model_name,
    evaluation_strategy="epoch",
    learning_rate=5e-05,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
    fp16=True,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    run_name=model_name
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
""" Step 3: Specify logic for computing ROUGE scores """

import evaluate
import numpy as np
from datasets import load_metric
from nltk.tokenize import sent_tokenize

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Compute ROUGE scores
    rouge_score_metric = evaluate.load('rouge')
    
    result = rouge_score_metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [14]:
""" Step 4: Instantiate Data Collator """

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [15]:
""" Step 5: Login to HF to log training and push to HF-Hub """

from huggingface_hub import notebook_login

notebook_login()

In [16]:
""" Step 6: Instantiate Trainer object """

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/adityashukzy/bart-base-arxiv-sum into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Download file pytorch_model.bin:   0%|          | 9.37k/532M [00:00<?, ?B/s]

Download file runs/Apr06_06-31-18_e4fe06ea411e/events.out.tfevents.1680762723.e4fe06ea411e.23.0: 100%|########…

...

Clean file training_args.bin:  28%|##7       | 1.00k/3.62k [00:00<?, ?B/s]

Clean file runs/Apr08_09-39-23_62b3516d853c/1680947179.0213175/events.out.tfevents.1680947179.62b3516d853c.24.…

Clean file runs/Apr08_09-39-23_62b3516d853c/events.out.tfevents.1680947178.62b3516d853c.24.0:  16%|#6        |…

Clean file pytorch_model.bin:   0%|          | 1.00k/532M [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [17]:
print_gpu_utilization()

GPU memory occupied: 1367 MB.


In [18]:
""" Step 8: Start training """
### wandb.ai token = 060c7b345b7e3d3b28e3729bc79127874fee1e84 ###

result = trainer.train(resume_from_checkpoint='/kaggle/working/bart-base-arxiv-sum')

You are resuming training from a checkpoint trained with 4.27.3 of Transformers but your current version is 4.27.4. This is not recommended and could yield to errors or unwanted behaviors.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


AttributeError: 'numpy.float64' object has no attribute 'mid'

In [ ]:
print_summary(result)

In [ ]:
""" Step 9 : Evaluate performance pre fine-tuning """

trainer.evaluate()

In [ ]:
""" Step 10: Tag as a Summarization model and push to HF Hub """

trainer.push_to_hub(commit_message="Training complete", tags="summarization")